In [1]:
# Path to code repository
code_repo_dir = '/Users/user/Documents/Codes/H5N1/H5N1-State-and-County-Simulations-and-Analysis/'

In [2]:
# Loading necessary packages
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FuncFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable

/Users/user/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# 1. Computing number of dairy cows in each dairy premise

In [3]:
# File paths. Change according to your local directory paths
input_sub_dir_1 = 'data/raw/'    
input_sub_dir_2 = 'data/processed/'

output_sub_dir_1 = 'data/processed/'
output_sub_dir_2 = 'outputs/figures/'

input_filename_1 = 'cattle_FLAPS_for_USAMMv3.txt'
input_filepath_1 = code_repo_dir + input_sub_dir_1 + input_filename_1


output_filename_1 = 'dairy_cows_by_premises.csv'
output_filepath_1 = code_repo_dir + output_sub_dir_1 + output_filename_1

county_shapefile_path = code_repo_dir + input_sub_dir_1 + 'gadm41_USA_shp/gadm41_USA_2.shp'
state_shapefile_path = code_repo_dir + input_sub_dir_1 + 'gadm41_USA_shp/gadm41_USA_1.shp'

In [4]:
# Load the FLAPS file that contain information of cattle per premises (eg. farm)
cattle_by_premises_df = pd.read_csv(input_filepath_1, sep = "\t")
cattle_by_premises_df[cattle_by_premises_df.type == "d"]

,Id,County,X,Y,Lat,Lon,type,b,d
53,757624,1001,844489,-8813162,32.590559,-86.803342,d,0,16
118,757844,1001,875641,-8810000,32.591639,-86.467205,d,0,13
126,757842,1001,875678,-8815462,32.543002,-86.472609,d,0,15
229,756096,1003,765523,-9041656,30.615888,-87.854986,d,0,4
245,756766,1003,799779,-9063597,30.393238,-87.517768,d,0,2
...,...,...,...,...,...,...,...,...,...
815615,729298,56043,-972175,-7522047,43.910056,-108.098869,d,0,2
815616,729318,56043,-964822,-7520318,43.933834,-108.010241,d,0,2
815749,730103,56045,-656624,-7562069,43.857095,-104.114942,d,0,4
815781,730094,56045,-655476,-7534925,44.100281,-104.130074,d,0,7


In [5]:
# Filter rows by cattle type 'd' (for dairy cow) and select the columns: Premises Id (Id), 
# County FIPS (County), Lat, Lon, Number of dairy cattle in premises (d) 
dairy_cows_by_premises_df = cattle_by_premises_df.loc[
    cattle_by_premises_df.type == "d",  ['Id', 'County', 'Lat', 'Lon', 'd']
].reset_index(drop=True)
dairy_cows_by_premises_df.to_csv(output_filepath_1, index=False)

# 2. Computing number of dairy cows by counties

In [6]:
input_filename_2 = 'dairy_cows_by_premises.csv'
input_filepath_2 = code_repo_dir + input_sub_dir_2 + input_filename_2

output_filename_2 = 'dairy_cows_by_counties.csv'
output_filepath_2 = code_repo_dir + output_sub_dir_1 + output_filename_2

print(input_filepath_2, output_filepath_2)

/Users/user/Documents/Codes/H5N1/H5N1-State-and-County-Simulations-and-Analysis/data/processed/dairy_cows_by_premises.csv /Users/user/Documents/Codes/H5N1/H5N1-State-and-County-Simulations-and-Analysis/data/processed/dairy_cows_by_counties.csv


In [7]:
dairy_cows_by_premises_df = pd.read_csv(input_filepath_2)
dairy_cows_by_premises_df

,Id,County,Lat,Lon,d
0,757624,1001,32.590559,-86.803342,16
1,757844,1001,32.591639,-86.467205,13
2,757842,1001,32.543002,-86.472609,15
3,756096,1003,30.615888,-87.854986,4
4,756766,1003,30.393238,-87.517768,2
...,...,...,...,...,...
63979,729298,56043,43.910056,-108.098869,2
63980,729318,56043,43.933834,-108.010241,2
63981,730103,56045,43.857095,-104.114942,4
63982,730094,56045,44.100281,-104.130074,7


In [8]:
# Load US shape file with counties as administrative regions and trim to mainland 
counties_shp_gdf = gpd.read_file(county_shapefile_path)
counties_shp_gdf = counties_shp_gdf.cx[-130:-60, 25:50]
counties_shp_gdf

,GID_2,GID_0,COUNTRY,GID_1,NAME_1,NL_NAME_1,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry
0,USA.1.1_1,USA,United States,USA.1_1,Alabama,NA,Autauga,NA,NA,County,County,NA,US.AL.AU,"POLYGON ((-86.91668 32.66431, -86.82678 32.660..."
1,USA.1.2_1,USA,United States,USA.1_1,Alabama,NA,Baldwin,NA,NA,County,County,NA,US.AL.BD,"MULTIPOLYGON (((-87.74416 30.29445, -87.74389 ..."
2,USA.1.3_1,USA,United States,USA.1_1,Alabama,NA,Barbour,NA,NA,County,County,NA,US.AL.BR,"POLYGON ((-85.41607 31.61962, -85.49133 31.620..."
3,USA.1.4_1,USA,United States,USA.1_1,Alabama,NA,Bibb,NA,NA,County,County,NA,US.AL.BI,"POLYGON ((-87.42078 32.87446, -87.42028 32.971..."
4,USA.1.5_1,USA,United States,USA.1_1,Alabama,NA,Blount,NA,NA,County,County,NA,US.AL.BU,"POLYGON ((-86.96287 33.85858, -86.96512 33.863..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3143,USA.51.19_1,USA,United States,USA.51_1,Wyoming,NA,Sweetwater,NA,NA,County,County,NA,US.WY.SW,"POLYGON ((-107.91540 41.00295, -107.96598 41.0..."
3144,USA.51.20_1,USA,United States,USA.51_1,Wyoming,NA,Teton,NA,NA,County,County,NA,US.WY.TE,"POLYGON ((-111.04362 43.31358, -111.04391 43.4..."
3145,USA.51.21_1,USA,United States,USA.51_1,Wyoming,NA,Uinta,NA,NA,County,County,NA,US.WY.UI,"POLYGON ((-111.04584 41.25143, -111.04594 41.3..."
3146,USA.51.22_1,USA,United States,USA.51_1,Wyoming,NA,Washakie,NA,NA,County,County,NA,US.WY.WA,"POLYGON ((-108.55055 44.16851, -107.14581 44.1..."


In [9]:
# Convert the dairy_cows_by_premises dataframe to a geodataframe
geometry = [Point(xy) for xy in zip(dairy_cows_by_premises_df['Lon'], dairy_cows_by_premises_df['Lat'])]
dairy_cows_by_premises_gdf = gpd.GeoDataFrame(dairy_cows_by_premises_df, geometry=geometry)
dairy_cows_by_premises_gdf

,Id,County,Lat,Lon,d,geometry
0,757624,1001,32.590559,-86.803342,16,POINT (-86.80334 32.59056)
1,757844,1001,32.591639,-86.467205,13,POINT (-86.46721 32.59164)
2,757842,1001,32.543002,-86.472609,15,POINT (-86.47261 32.54300)
3,756096,1003,30.615888,-87.854986,4,POINT (-87.85499 30.61589)
4,756766,1003,30.393238,-87.517768,2,POINT (-87.51777 30.39324)
...,...,...,...,...,...,...
63979,729298,56043,43.910056,-108.098869,2,POINT (-108.09887 43.91006)
63980,729318,56043,43.933834,-108.010241,2,POINT (-108.01024 43.93383)
63981,730103,56045,43.857095,-104.114942,4,POINT (-104.11494 43.85710)
63982,730094,56045,44.100281,-104.130074,7,POINT (-104.13007 44.10028)


In [10]:
# Set the CRS of the cattle data to match the states shapefile if they are different
# Assuming states_gdf is in EPSG:4326, update if necessary
dairy_cows_by_premises_gdf.set_crs(epsg=4326, inplace=True)
dairy_cows_by_premises_gdf.to_crs(counties_shp_gdf.crs, inplace=True)

In [11]:
# Perform a spatial join to aggregate cows data by counties
# Spatial join to get county names
dairy_cows_by_premises_with_county_gdf = gpd.sjoin(dairy_cows_by_premises_gdf, counties_shp_gdf, how='left', op='within')
dairy_cows_by_premises_with_county_gdf

/Users/user/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3466: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,Id,County,Lat,Lon,d,geometry,index_right,GID_2,GID_0,COUNTRY,GID_1,NAME_1,NL_NAME_1,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2
0,757624,1001,32.590559,-86.803342,16,POINT (-86.80334 32.59056),0.0,USA.1.1_1,USA,United States,USA.1_1,Alabama,NA,Autauga,NA,NA,County,County,NA,US.AL.AU
1,757844,1001,32.591639,-86.467205,13,POINT (-86.46721 32.59164),0.0,USA.1.1_1,USA,United States,USA.1_1,Alabama,NA,Autauga,NA,NA,County,County,NA,US.AL.AU
2,757842,1001,32.543002,-86.472609,15,POINT (-86.47261 32.54300),0.0,USA.1.1_1,USA,United States,USA.1_1,Alabama,NA,Autauga,NA,NA,County,County,NA,US.AL.AU
3,756096,1003,30.615888,-87.854986,4,POINT (-87.85499 30.61589),1.0,USA.1.2_1,USA,United States,USA.1_1,Alabama,NA,Baldwin,NA,NA,County,County,NA,US.AL.BD
4,756766,1003,30.393238,-87.517768,2,POINT (-87.51777 30.39324),1.0,USA.1.2_1,USA,United States,USA.1_1,Alabama,NA,Baldwin,NA,NA,County,County,NA,US.AL.BD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63979,729298,56043,43.910056,-108.098869,2,POINT (-108.09887 43.91006),3146.0,USA.51.22_1,USA,United States,USA.51_1,Wyoming,NA,Washakie,NA,NA,County,County,NA,US.WY.WA
63980,729318,56043,43.933834,-108.010241,2,POINT (-108.01024 43.93383),3146.0,USA.51.22_1,USA,United States,USA.51_1,Wyoming,NA,Washakie,NA,NA,County,County,NA,US.WY.WA
63981,730103,56045,43.857095,-104.114942,4,POINT (-104.11494 43.85710),3147.0,USA.51.23_1,USA,United States,USA.51_1,Wyoming,NA,Weston,NA,NA,County,County,NA,US.WY.WE
63982,730094,56045,44.100281,-104.130074,7,POINT (-104.13007 44.10028),3147.0,USA.51.23_1,USA,United States,USA.51_1,Wyoming,NA,Weston,NA,NA,County,County,NA,US.WY.WE


In [12]:
# Aggregate the number of dairy cows by county and number of unique premises in each county
dairy_cows_by_county = (
    dairy_cows_by_premises_with_county_gdf
    .groupby(['NAME_1', 'NAME_2'])
    .agg(
        d=('d', 'sum'),     # sum of cows
        p=('Id', 'nunique')  # count of unique premises
    )
    .reset_index()
)

dairy_cows_by_county.sort_values(by = 'd', ascending = False)

,NAME_1,NAME_2,d,p
144,California,Tulare,891885,256
117,California,Merced,519806,229
140,California,Stanislaus,327318,238
376,Idaho,Gooding,325580,64
109,California,Kings,307048,111
...,...,...,...,...
728,Kansas,Ottawa,2,1
1533,North Carolina,Pamlico,2,1
95,California,Alameda,2,1
96,California,Amador,2,1


In [13]:
dairy_cows_by_county.to_csv(output_filepath_2, index=False)

# 3. Plotting county-wise dairy cows distribution

In [14]:
# Merge the aggregated data with the counties GeoDataFrame
counties_dc_geom_gdf = counties_shp_gdf.merge(dairy_cows_by_county, 
                                        left_on=['NAME_1', 'NAME_2'], right_on=['NAME_1','NAME_2'], how='left')

counties_dc_geom_gdf['p'] = counties_dc_geom_gdf['p'].fillna(0).astype(int)


counties_dc_geom_gdf

,GID_2,GID_0,COUNTRY,GID_1,NAME_1,NL_NAME_1,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry,d,p
0,USA.1.1_1,USA,United States,USA.1_1,Alabama,NA,Autauga,NA,NA,County,County,NA,US.AL.AU,"POLYGON ((-86.91668 32.66431, -86.82678 32.660...",44.0,3
1,USA.1.2_1,USA,United States,USA.1_1,Alabama,NA,Baldwin,NA,NA,County,County,NA,US.AL.BD,"MULTIPOLYGON (((-87.74416 30.29445, -87.74389 ...",120.0,6
2,USA.1.3_1,USA,United States,USA.1_1,Alabama,NA,Barbour,NA,NA,County,County,NA,US.AL.BR,"POLYGON ((-85.41607 31.61962, -85.49133 31.620...",13.0,2
3,USA.1.4_1,USA,United States,USA.1_1,Alabama,NA,Bibb,NA,NA,County,County,NA,US.AL.BI,"POLYGON ((-87.42078 32.87446, -87.42028 32.971...",NaN,0
4,USA.1.5_1,USA,United States,USA.1_1,Alabama,NA,Blount,NA,NA,County,County,NA,US.AL.BU,"POLYGON ((-86.96287 33.85858, -86.96512 33.863...",27.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3112,USA.51.19_1,USA,United States,USA.51_1,Wyoming,NA,Sweetwater,NA,NA,County,County,NA,US.WY.SW,"POLYGON ((-107.91540 41.00295, -107.96598 41.0...",29.0,6
3113,USA.51.20_1,USA,United States,USA.51_1,Wyoming,NA,Teton,NA,NA,County,County,NA,US.WY.TE,"POLYGON ((-111.04362 43.31358, -111.04391 43.4...",NaN,0
3114,USA.51.21_1,USA,United States,USA.51_1,Wyoming,NA,Uinta,NA,NA,County,County,NA,US.WY.UI,"POLYGON ((-111.04584 41.25143, -111.04594 41.3...",4.0,2
3115,USA.51.22_1,USA,United States,USA.51_1,Wyoming,NA,Washakie,NA,NA,County,County,NA,US.WY.WA,"POLYGON ((-108.55055 44.16851, -107.14581 44.1...",22.0,9


In [15]:
fig_filename = 'dairy_cows_by_county_logscale.png'
fig_path = code_repo_dir + output_sub_dir_2 + fig_filename
print(fig_path)

/Users/user/Documents/Codes/H5N1/H5N1-State-and-County-Simulations-and-Analysis/outputs/figures/dairy_cows_by_county_logscale.png


In [16]:
counties_dc_geom_gdf['log_d'] = np.log(counties_dc_geom_gdf['d'])
counties_dc_geom_gdf

,GID_2,GID_0,COUNTRY,GID_1,NAME_1,NL_NAME_1,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry,d,p,log_d
0,USA.1.1_1,USA,United States,USA.1_1,Alabama,NA,Autauga,NA,NA,County,County,NA,US.AL.AU,"POLYGON ((-86.91668 32.66431, -86.82678 32.660...",44.0,3,3.784190
1,USA.1.2_1,USA,United States,USA.1_1,Alabama,NA,Baldwin,NA,NA,County,County,NA,US.AL.BD,"MULTIPOLYGON (((-87.74416 30.29445, -87.74389 ...",120.0,6,4.787492
2,USA.1.3_1,USA,United States,USA.1_1,Alabama,NA,Barbour,NA,NA,County,County,NA,US.AL.BR,"POLYGON ((-85.41607 31.61962, -85.49133 31.620...",13.0,2,2.564949
3,USA.1.4_1,USA,United States,USA.1_1,Alabama,NA,Bibb,NA,NA,County,County,NA,US.AL.BI,"POLYGON ((-87.42078 32.87446, -87.42028 32.971...",NaN,0,NaN
4,USA.1.5_1,USA,United States,USA.1_1,Alabama,NA,Blount,NA,NA,County,County,NA,US.AL.BU,"POLYGON ((-86.96287 33.85858, -86.96512 33.863...",27.0,2,3.295837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3112,USA.51.19_1,USA,United States,USA.51_1,Wyoming,NA,Sweetwater,NA,NA,County,County,NA,US.WY.SW,"POLYGON ((-107.91540 41.00295, -107.96598 41.0...",29.0,6,3.367296
3113,USA.51.20_1,USA,United States,USA.51_1,Wyoming,NA,Teton,NA,NA,County,County,NA,US.WY.TE,"POLYGON ((-111.04362 43.31358, -111.04391 43.4...",NaN,0,NaN
3114,USA.51.21_1,USA,United States,USA.51_1,Wyoming,NA,Uinta,NA,NA,County,County,NA,US.WY.UI,"POLYGON ((-111.04584 41.25143, -111.04594 41.3...",4.0,2,1.386294
3115,USA.51.22_1,USA,United States,USA.51_1,Wyoming,NA,Washakie,NA,NA,County,County,NA,US.WY.WA,"POLYGON ((-108.55055 44.16851, -107.14581 44.1...",22.0,9,3.091042


In [ ]:
# Plot the states with colors corresponding to the number of dairy cows
fig, ax = plt.subplots(figsize=(12, 10))
counties_dc_geom_gdf.boundary.plot(ax=ax, linewidth=1, color='black', alpha = 0.2)
state_borders = gpd.read_file(state_shapefile_path)
state_borders = state_borders.cx[-130:-60, 25:50]

# Plot state borders
state_borders.plot(ax=plt.gca(), color='none', edgecolor='black', linewidth=1.5, alpha =1)

# Plot the aggregated dairy cows count per county
counties_dc_geom_gdf.plot(column='log_d', ax=ax, legend=True, 
                legend_kwds={'shrink': 0.4, 'label': "Dairy Cows per County (log-scale)",
                             'orientation': "horizontal", 
                             'pad': 0.1, 'fmt': '%.0f'},
                cmap = "OrRd"
        )

cbar = ax.get_figure().axes[-1] 
cbar.set_xlabel("Dairy Cows per County (log-scale)", fontsize=12)  # Adjust fontsize here

plt.title('Population of Dairy Cows by County in the US (log scale)', fontsize = 18)
plt.xlabel('Longitude', fontsize = 14)
plt.ylabel('Latitude', fontsize = 14)
plt.savefig(fig_path, dpi = 600)
plt.show()

# 4. Computing number of dairy cows by state

In [ ]:
output_filename_3 = 'dairy_cows_by_state.csv'
output_filepath_3 = code_repo_dir + output_sub_dir_1 + output_filename_3
output_filepath_3

In [ ]:
# dairy_cows_by_state = counties_dc_geom_gdf.groupby('NAME_1')['d'].sum().reset_index()
dairy_cows_by_state = (
    counties_dc_geom_gdf
    .groupby(['NAME_1'])
    .agg(
        d=('d', 'sum'),     # sum of cows
        p=('p', 'sum')  # count of unique premises
    )
    .reset_index()
)

dairy_cows_by_state["d"] = dairy_cows_by_state["d"].astype(int)
dairy_cows_by_state.sort_values(by = 'd', ascending = False)

In [ ]:
dairy_cows_by_state.to_csv(output_filepath_3, index=False)